In [ ]:
# !pip install transformers
# !pip install pyserini

In [3]:
# import os

# os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/jdk-13.0.2.jdk/Contents/Home'

In [57]:
from matplotlib import pyplot as plot

In [29]:
import torch

In [61]:
import numpy

In [11]:
from transformers import *

tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')

In [7]:
from pyserini.search import pysearch

In [8]:
searcher = pysearch.SimpleSearcher('./lucene-index-covid/')

In [145]:
query = 'pneumonia coronavirus viral'

In [146]:
hits = searcher.search(query)

In [147]:
def relevant_snippets(query_ids, query_state, abstract_ids, abstract_state, 
                      n_abstract_words = 5, window = 5, prefix='\t'):
    query_state = query_state / (query_state ** 2).sum(-1, keepdims=True)
    abstract_state = abstract_state / (abstract_state ** 2).sum(-1, keepdims=True)

    query_state_ = query_state.view(1, query_state.size(1), 1, query_state.size(2))
    abstract_state_ = abstract_state.view(1, 1, abstract_state.size(1), abstract_state.size(2))
    query_abstract_match = (query_state_ * abstract_state_).sum(-1)

    qa_match = query_abstract_match.data.numpy()[0]

    abstract_match = qa_match.sum(0)
    abstract_words_idx = numpy.argsort(abstract_match)[-n_abstract_words:][::-1]

    abstract_words = tokenizer.convert_ids_to_tokens(abstract_ids[0])[1:-1]
    important_words = [abstract_words[idx] for idx in abstract_words_idx]

    query_words = tokenizer.convert_ids_to_tokens(query_ids[0])[1:-1]

    for word_idx, word in zip(abstract_words_idx,important_words):
        best_qword = numpy.argmax(qa_match[:, word_idx])
        surround_text_l = ' '.join(abstract_words[word_idx-window:word_idx])
        surround_text_r = ' '.join(abstract_words[word_idx+1:word_idx+window])
        surround_text = '... ' + surround_text_l + F' $${abstract_words[word_idx]}$$ ' + surround_text_r + ' ...'
        surround_text = surround_text.replace(' ##', '')
        print(F'{prefix}{query_words[best_qword]}: {surround_text}')

In [148]:
query_ids = torch.tensor([tokenizer.encode(query, add_special_tokens=True)])

with torch.no_grad():
    query_state = model(query_ids)[0]
    query_state = query_state[:, 1:-1, :]

In [156]:
n_documents = 10

for ii in range(0, n_documents):
    print(f'{i+1} {hits[i].docid} {hits[i].score} {searcher.doc(hits[i].ldocid).object.getField("title").stringValue()}')
    
    doc = searcher.doc(hits[ii].ldocid).object
    content = hits[0].content # raw content

    abstract_ids = torch.tensor([tokenizer.encode(doc.getField('abstract').stringValue(), add_special_tokens=True)])
    abstract_ids = abstract_ids[0,:512].unsqueeze(0)
    with torch.no_grad():
        abstract_state = model(abstract_ids)[0]
        abstract_state = abstract_state[:, 1:-1, :]
    
    relevant_snippets(query_ids, query_state, abstract_ids, abstract_state, window=7)

10 43946 2.6057000160217285 A familial cluster of pneumonia associated with the 2019 novel coronavirus indicating person-to-person transmission: a study of a family cluster
	viral: ... secondary to respiratory viral infections other than $$influenza$$ remain unknown . the purpose of ...
	pneumonia: ... common in these patients . in contrast $$,$$ coronavirus , parainflu ...
	viral: ...eumovirus infections was caused by $$similar$$ bacteria , and the findings indicated ...
	corona: ... in these patients . in contrast , $$corona$$virus , parainfluenza ...
	corona: ...1 ) : pneumonia in patients with $$influenza$$ virus ( type a / b ...
10 43946 2.6057000160217285 A familial cluster of pneumonia associated with the 2019 novel coronavirus indicating person-to-person transmission: a study of a family cluster
	pneumonia: ... , coronavirus 229e and oc $$##43$$ , chlamydia pneumoniae and myco ...
	viral: ... adenovirus , coronavirus 229e and $$oc$$43 , chlamydia pneumoniae and ...
	viral: ... 